<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Sum-of-all-amplitudes-for-all-inputs" data-toc-modified-id="Sum-of-all-amplitudes-for-all-inputs-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Sum of all amplitudes for all inputs</a></span></li><li><span><a href="#Sum-of-amplitudes-for-single-input" data-toc-modified-id="Sum-of-amplitudes-for-single-input-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sum of amplitudes for single input</a></span></li><li><span><a href="#Single-amplitude-for-single-input" data-toc-modified-id="Single-amplitude-for-single-input-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Single amplitude for single input</a></span></li><li><span><a href="#Simulate-all-output-states-for-given-input" data-toc-modified-id="Simulate-all-output-states-for-given-input-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Simulate all output states for given input</a></span><ul class="toc-item"><li><span><a href="#First-error.-Bucket-order" data-toc-modified-id="First-error.-Bucket-order-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>First error. Bucket order</a></span></li><li><span><a href="#Second-error.-Variable-relabel" data-toc-modified-id="Second-error.-Variable-relabel-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Second error. Variable relabel</a></span></li><li><span><a href="#Correct-usage" data-toc-modified-id="Correct-usage-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Correct usage</a></span></li></ul></li></ul></div>

In [92]:
import numpy as np

import qtree
from qtree.operators import Gate

In [93]:
help(Gate)

Help on class Gate in module qtree.operators:

class Gate(builtins.object)
 |  Gate(*qubits)
 |  
 |  Base class for quantum gates.
 |  
 |  Attributes
 |  ----------
 |  name: str
 |          The name of the gate
 |  
 |  parameters: dict
 |           Parameters used by the gate (may be empty)
 |  
 |  qubits: tuple
 |          Qubits the gate acts on
 |  
 |  changed_qubits : tuple
 |          Tuple of ints which states what qubit's bases are changed
 |          (along which qubits the gate is not diagonal).
 |  
 |  cirq_op: Cirq.GridQubit
 |          Cirq 2D gate. Used for unit tests. Optional
 |  
 |  Methods
 |  -------
 |  gen_tensor(): numpy.array
 |          The gate tensor. For each qubit a gate
 |          either introduces a new variable (non-diagonal gate, like X)
 |          or does not (diagonal gate, like T). Multiqubit gates
 |          can be diagonal on some of the variables, and not diagonal on
 |          others (like ccX). The order of dimensions IS ALWAYS
 |     

In [94]:
class MyGate(Gate):
    name = 'MyGate'
    _changes_qubits=(0,)
    def gen_tensor(self):
        tensor = 1/np.sqrt(2)*np.array([
            [1,1]
            ,[1,-1]
        ])
        return tensor
    
myGate = MyGate(0)
myGate

MyGate(0)

In [95]:
from qtree import optimizer

tensor_expr, data_dict, bra, ket = optimizer.circ2buckets(1, [[myGate]])
print(tensor_expr)
print(data_dict)
print(bra)
print(ket)

[[M(o_0,v_1)], [MyGate(v_1,v_2)], [M(v_2,i_0)], []]
{('M', -6923940778079747740): array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]], dtype=complex64), ('MyGate', -7879287038868799089): array([[ 0.70710678,  0.70710678],
       [ 0.70710678, -0.70710678]])}
[o_0]
[i_0]


In [96]:
from qtree import np_framework as npfr

## Sum of all amplitudes for all inputs

This is just a full contraction of the tensor network

In [97]:
numpy_buckets = npfr.get_sliced_np_buckets(tensor_expr
                                           ,data_dict
                                           ,{}
                                          )
numpy_buckets

[[M(v_0,v_1)], [MyGate(v_1,v_2)], [M(v_2,v_3)], []]

In [98]:
result = optimizer.bucket_elimination(numpy_buckets, npfr.process_bucket_np)
result

E3()

In [99]:
result.data

(1.414213562373095+0j)

## Sum of amplitudes for single input

This is a contraction of a network that was sliced over input indices

In [100]:
initial_state = 0
slice_dict = qtree.utils.slice_from_bits(initial_state, ket)
slice_dict

{i_0: slice(0, 1, None)}

In [101]:
numpy_buckets = npfr.get_sliced_np_buckets(
    tensor_expr, data_dict, slice_dict
)
print(numpy_buckets)
print("Output tensor:",numpy_buckets[0][0].data)
print("Input tensor:", numpy_buckets[-2][0].data)
result = optimizer.bucket_elimination(numpy_buckets, npfr.process_bucket_np)
result.data

[[M(v_0,v_1)], [MyGate(v_1,v_2)], [M(v_2,v_3)], []]
Output tensor: [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Input tensor: [[1.+0.j]
 [0.+0.j]]


(1.414213562373095+0j)

## Single amplitude for single input

In [102]:
initial_state = 0
output_state = 0
slice_dict = qtree.utils.slice_from_bits(initial_state, ket)
slice_dict.update( qtree.utils.slice_from_bits(output_state, bra) )
slice_dict

{i_0: slice(0, 1, None), o_0: slice(0, 1, None)}

In [103]:
def simulate_buckets(tensor_expr
                     , data_dict, slice_dict):
    numpy_buckets = npfr.get_sliced_np_buckets(
        tensor_expr, data_dict, slice_dict
    )
    print(numpy_buckets)
    print("Output tensor:",numpy_buckets[0][0].data)
    print("Input tensor:", numpy_buckets[-2][0].data)
    result = optimizer.bucket_elimination(numpy_buckets, npfr.process_bucket_np)
    return result.data

In [104]:
output = simulate_buckets(tensor_expr, data_dict, slice_dict)
output

[[M(v_0,v_1)], [MyGate(v_1,v_2)], [M(v_2,v_3)], []]
Output tensor: [[1.+0.j 0.+0.j]]
Input tensor: [[1.+0.j]
 [0.+0.j]]


(0.7071067811865475+0j)

## Simulate all output states for given input

This is a partial contraction,
where we leave out the latest, output index

In [105]:
initial_state = 0
slice_dict = qtree.utils.slice_from_bits(initial_state, ket)
#slice_dict.update({var: slice(None) for var in bra})

### First error. Bucket order

In [106]:
numpy_buckets = npfr.get_sliced_np_buckets(
    tensor_expr, data_dict, slice_dict
)

print(numpy_buckets)
print("Output tensor:",numpy_buckets[0][0].data)
print("Input tensor:", numpy_buckets[-2][0].data)
print("Input tensor vars:", numpy_buckets[-2][0].indices)
result = optimizer.bucket_elimination(numpy_buckets
                                      , npfr.process_bucket_np
                                      , n_var_nosum=1
                                     )
result.data

[[M(v_0,v_1)], [MyGate(v_1,v_2)], [M(v_2,v_3)], []]
Output tensor: [[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Input tensor: [[1.+0.j]
 [0.+0.j]]
Input tensor vars: (v_2, v_3)


array([1.41421356+0.j])

Wrong result! Because the ordering is inverse. We first contracted our input variable, which was already sliced to the first state

### Second error. Variable relabel

In [107]:
reversed_buckets = list(reversed(numpy_buckets))

result = optimizer.bucket_elimination(reversed_buckets
                                      , npfr.process_bucket_np
                                      , n_var_nosum=1
                                     )
result.data

ValueError: Index mismatch in __mul__: () times (v_0, v_1, v_3)

### Correct usage

Have to reorder the buckets. From the **original** buckets, not numpy

In [133]:
all_tensors = sum(tensor_expr, [])
print(all_tensors)
all_vars = set(sum([tensor.indices for tensor in all_tensors], tuple() ))
all_vars

[M(o_0,v_1), MyGate(v_1,v_2), M(v_2,i_0)]


{o_0, v_1, v_2, i_0}

In [121]:
peo = list(all_vars - set(bra))
peo

[v_1, i_0, v_2]

In [127]:
perm_buckets, perm_dict = qtree.optimizer.reorder_buckets(
    tensor_expr, peo + bra
)

ket_vars = sorted([perm_dict[idx] for idx in ket], key=str)
bra_vars = sorted([perm_dict[idx] for idx in bra], key=str)

initial_state = 0
slice_dict = qtree.utils.slice_from_bits(initial_state, ket_vars)

In [132]:

numpy_buckets = npfr.get_sliced_np_buckets(
    perm_buckets, data_dict, slice_dict
)

print(numpy_buckets)
result = optimizer.bucket_elimination(numpy_buckets
                                      , npfr.process_bucket_np
                                      , n_var_nosum=1
                                     )
result.data

[[M(v_0,v_3), MyGate(v_0,v_2)], [M(v_1,v_2)], [], []]


array([0.70710678+0.j, 0.70710678+0.j])